# ※ AWS는 적재, 조회 등에 비용이 발생하니 최종 코드가 아닌 한 local에서 TEST!

In [ ]:
# aws 계정 로그인 셀
import pymysql
import pandas as pd # 불요. 출력값 보기용.

# 계정 정보 파일에서 필요 인자 딕셔너리로 생성
with open('secret_local_db.config') as f:
    res = dict(map(lambda x: x.replace('\n','').split('='), f.readlines()))
    for i, v in res.items():
        if v.isdigit(): res[i] = int(v)
# 로그인 위한 인자명 출력
print(res.keys())
# 딕셔너리 언패킹하여 인자 값 할당 후 서버 연동
remote = pymysql.connect(**res)

In [ ]:
# db 및 유저 생성

# with로 여는 것은 cursor가 여러 개 열리는 것을 예방하기 위함.
with remote.cursor() as cur:
    # db명 설정 및 생성
    dbname = 'db4_6'
    # cur.execute(f'create database {dbname}')
    cur.execute('show databases')
    # fetchall(): 출력문 전체 가져오기
    # print(cur.fetchall())
    print(f'{"  show databases  ":=^80}')
    print(pd.DataFrame([_ for _ in cur.fetchall()], columns=['database_name']))
    
    # # user 생성 및 권한 지정
    # username = 'tmp_01'
    # password = 'tmp_1234'   # 연습용이 아닌 실제 생성 user는 secret 파일을 통해 인자 입력 필요
    # cur.execute('use mysql')
    # # user 생성, %는 외부 접속 사용자라는 의미.
    # cur.execute(f'create user \'{username}\'@\'%\' identified by \'{password}\'')
    # # 특정 db 권한 전부(*) 부여
    # cur.execute(f'grant all on {dbname}.* to \'{username}\'@\'%\'')
    # # 특정 user 권한 확인
    # cur.execute(f'show grants for \'{username}\'@\'%\'')
    # # print(cur.fetchall())
    # print(f'''{"  show grants for '{username}'@'%'  ":=^80}''')
    # print(pd.DataFrame([_ for _ in cur.fetchall()], columns=['grant users']))

In [3]:
# # db 및 유저 삭제

# with remote.cursor() as cur:
#     # user 권한 제거 및 user 삭제
#     username = 'tmp_01'
#     cur.execute('use mysql')
#     # user 테이블의 host, user 컬럼 조회
#     cur.execute('select host, user from user')
#     # print(cur.fetchall())
#     print(f'{"  select host, user from user  ":=^80}')
#     print(pd.DataFrame([_ for _ in cur.fetchall()], columns=['host', 'user']))
    
#     # 특정 db 권한 전부(*) 제거
#     cur.execute(f'revoke all on {dbname}.* from \'{username}\'@\'%\'')
#     # 특정 user 권한 확인
#     cur.execute(f'show grants for \'{username}\'@\'%\'')
#     # print(cur.fetchall())
#     print(f'''{"  show grants for '{username}'@'%'  ":=^80}''')
#     print(pd.DataFrame([_ for _ in cur.fetchall()]))

#     # 특정 user 삭제
#     cur.execute(f'drop user \'{username}\'@\'%\'')
#     # user 제거 확인
#     cur.execute('select host, user from user')
#     # print(cur.fetchall())
#     print(f'{"  select host, user from user  ":=^80}')
#     print(pd.DataFrame([_ for _ in cur.fetchall()], columns=['host', 'user']))


#     # db 삭제
#     cur.execute('show databases')
#     # print(cur.fetchall())
#     print(f'{"  show databases  ":=^80}')
#     print(pd.DataFrame([_ for _ in cur.fetchall()], columns=['database_name']))
    
#     dbname = 'db4_6'
#     cur.execute(f'drop database {dbname}')
#     # db 삭제 확인
#     cur.execute('show databases')
#     # print(cur.fetchall())
#     print(f'{"  show databases  ":=^80}')
#     print(pd.DataFrame([_ for _ in cur.fetchall()], columns=['database_name']))

In [ ]:
with remote.cursor() as cur:
    cur.execute('show tables')
    print(f'{"  show tables  ":=^80}')
    print(pd.DataFrame([_ for _ in cur.fetchall()], columns=['tables']))

In [ ]:
sql_file = 'Elementary_ERD.sql'
# sql_file = 'Elementary_ERD.txt'

with remote.cursor() as cur:
    # 파일에서 '\ufeff'가 읽힐 경우 encoding하거나 replace로 제거
    with open(sql_file, 'r', encoding='utf-8-sig') as f :
        # split(';')이기에 마지막에 ['']가 존재하여 [:-1]로 슬라이싱
        commands = f.read().split(';')[:-1]
    for command in commands:
        cur.execute(command.strip())

In [2]:
import json

with remote.cursor() as cur:
    # category 불러오기
    try:
        with open('category.json', 'r') as f:
            category = json.load(f)
    except:
        print("category.json don't exist or path is worng.")

    # category 확인 및 적재
    done_list = []
    error_list = []
    try:
        cur.execute('select count(*) from category')
        if cur.fetchall()[0][0]==145:
            print('='*50)
            print('already all record is loaded on CATEGORY table')
            pass
        else:
            for platfrom in category.keys():
                for cat_1 in category[platfrom].keys():
                    for name, id in category[platfrom][cat_1].items():
                        try:
                            cur.execute(f"insert into category values({id}, '{cat_1}','{name}', '{platfrom}')")
                            done_list.append([id, cat_1, name, platfrom])
                        except:
                            error_list.append([id, cat_1, name, platfrom])
                            print(f"already values({id}, '{cat_1}','{name}', '{platfrom}' exist")
            print('='*50)
            print(f'done_tasl: {len(done_list)}, error_task: {len(error_list)}')
    except:
        print("make tables first!")

# DML은 별도 commit 필요!
remote.commit()
print('task is done!')